# Exercises Evaluation Metrics

In [2]:
import pandas as pd                                                     # Data manipulation
import numpy as np                                                      # Scientific computing
import matplotlib.pyplot as plt                                         # Data visualization
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis    # Discriminant Analysis


## Theoretical questions

### Question 1:
- We willen gaan voorspellen of studenten heel gemotiveerd zijn om een diploma Toegepaste Informatica te behalen.
    - De onafhankelijke gegevens (de voorspellers) zijn "het aantal broers en/of zussen", "aantal samenzweringen" en "het aantal uren wiskunde". Plaats de onafhankelijke variabelen in een nieuw dataframe met naam df en zorg ervoor dat de waarden van de voorspellers numeriek zijn en dat eventueel onrealistische gegevens worden omgezet in nan-waarden.
    - In de vragenlijst wil je het "belang van informatica" gaan voorspellen. Dit is een ordinaal gegeven dat je zal moeten reduceren tot twee mogelijke waarden: alles gelijk of meer dan "groot" is "groot", alles kleiner is "matig". Voeg het resultaat als een kolom toe aan het dataframe df.
    - Gebruik lineaire discriminant analyse als model om de afhankelijke variabele "belang van Informatica" te voorspellen met behulp van de onafhankelijke variabelen (voorspellers). Maak de voorspellingen voor de beschikbare gegevens.
    - Bestudeer de resultaten van de evaluatie metrieken:
        - Maak de confusion matrix
        - Bereken de accuracy, precision, recall en F1
        - Maak de ROC-curve

In [3]:
data1 = pd.read_csv('../Data/Data1.csv', delimiter=';', decimal=',')
display(data1.head())

,gapminder,schrijfhand,schoenmaat,lengte,bloedgroep,resus,siblings,Samenzweringen,rijbewijs,domicilie in Antwerpen,...,M1 - stelen,M2 - team,M3 - beschermen,M4 - voorbeeld,M5 - embryo,M6 - vrijheid,mascotte,zwaarste vak,boeienste vak,studiepunt
0,7/18,Rechts,43.0,184,O,positief,5,De tsunami van 2004 in de Indische Oceaan was ...,Categorie B: Wagens.,"buiten de stad (een dorp, een gehucht, het pla...",...,7,2,8,6,5,4.0,"1=Bever,2=Wasbeer,3=Otter,4=Wolf,5=Poema,6=Pan...",Management 1,Data science 1,16-20 uren
1,3/18,Rechts,42.0,163,Weet ik niet,Weet ik niet,1,Prinses Diana kwam niet per ongeluk om het leven,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,10,5,10,6,7,3.0,"1=Bever,2=Wolf,3=Bizon,4=Otter,5=Panter,6=Poem...",Management 1,Computersystemen 1,<= 10 uren
2,12/18,Rechts,45.0,183,AB,positief,2,Prinses Diana kwam niet per ongeluk om het lev...,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,6,3,7,7,10,6.0,"1=Bruine beer,2=Poema,3=Panter,4=Wolf,5=Vos,6=...",Management 1,Programmeren 1 - Java,16-20 uren
3,5/18,Rechts,44.0,175,A,positief,2,De wereld wordt eigenlijk gecontroleerd door e...,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,9,10,10,10,10,0.0,"1=Wasbeer,2=Wolf,3=Bruine beer,4=Das,5=Vos,6=B...",Management 1,Data science 1,<= 10 uren
4,6/18,Rechts,43.0,173,B,positief,3,De wereld wordt eigenlijk gecontroleerd door e...,Categorie B: Wagens.,in een stad,...,10,6,10,7,7,2.0,"1=Wolf,2=Vos,3=Bruine beer,4=Bizon,5=Panter,6=...",Management 1,Data science 1,<= 10 uren


### Question 2:
- In de tabel hieronder vind je de confusion matrix van een binaire classifier.

|            | Predicted YES | Predicted NO | Total |
|------------|---------------|--------------|-------|
| Actual YES | 100           | 5            | 105   |
| Actual NO  | 10            | 50           | 60    |
| Total      | 110           | 55           | 165   |

- Beantwoord nu de volgende vragen:
    - Wat zijn de waarden voor TP, TN, FP, FN?
    - Bereken nu de accuracy, precision en recall.
    - Bereken de F1 en F1.5-measures.
    - Wat is de TPR en FPR? Vergelijk met je recall. Wat stel je vast?

Schrijf functies in Python die bovenstaande metrieken kunnen berekenen uit een confusion matrix. Let op: dit is niet mogelijk met bestaande functies van sklearn.metrics, omdat deze werken op de ruwe data en niet op een reeds bestaande verwarringsmatrix.

- Maak van bovenstaande tabel een (confusion) matrix in Python.
- Schrijf een aparte functie voor elk metriek en controleer je oplossing. Hint: kijk even naar de volgende Python functie: np.diag.

### Question 3:
- In de tabel hieronder vind je de confusion matrix van een binaire classifier.

|            | Predicted YES | Predicted NO | Total |
|------------|---------------|--------------|-------|
| Actual A   | 100           | 0            | 100   |
| Actual B   | 50            | 5            | 55    |
| Total      | 150           | 5            | 155   |

- Beantwoord nu de volgende vragen:
- Wat zijn de waarden voor TP, TN, FP, FN?
- Bereken met je zelfgeschreven Python-functies de accuracy, precision en recall, en F1.
- Is dit een goede classifier?


### Question 4:
- In de tabel hieronder vind je de confusion matrix van een multiclass classifier.

|            | Asfalt | Beton | Gras | Boom | Gebouw | Total |
|------------|--------|-------|------|------|--------|-------|
| Asfalt     | 2385   | 0     | 0    | 0    | 12     | 2397  |
| Beton      | 4      | 332   | 1    | 0    | 0      | 337   |
| Gras       | 0      | 0     | 908  | 0    | 0      | 908   |
| Boom       | 1      | 0     | 8    | 1084 | 6      | 1099  |
| Gebouw     | 4      | 1     | 0    | 9    | 2053   | 2067  |
| Total      | 2394   | 333   | 917  | 1093 | 2071   | 6808  |

- Beantwoord nu de volgende vragen:
    - Wat zijn de waarden voor TP, FP, FN voor de klasse Gras?
Pas je functies voor alle evaluatiemetrieken aan zodat ze de waarde per klasse teruggeven.
    - Bereken nu de accuracy, precision en recall per klasse met deze functies.
    - Is dit een goede classifier?


### Question 5:
- Lees de data in het bestand `simpsons_roc1`. 
    - Gebruik de plot_roc functie (zie slides) om de ROC-curve te tekenen.
    - Wat is de beste treshold-waarde?
    - Wat is de AUC? Wat betekent het?
    - Pas de gevonden treshold-waarde (zie b.) toe op de kolom y_score (deze kolom bevat de resultaten van een neuraal netwerk predict_proba-functie -zie later-) om een nieuwe voorspelde kolom te maken.
    Voer de stappen a. tot en met d. uit met de dataset in het bestand "simpsons_roc2". Merk je 
    verschillen?


### Question 6:
- Twee lineaire discriminante analysemodellen werden gebouwd met de dataset-infert -zie bestand infert.csv- De resultaten van de voorspellingen zijn te vinden in de bestanden `education_roc1.csv` en `education_roc2.csv`.
    - Laad beide datasets
    - Gebruik de plot_roc functie (zie slides) om beide ROC-curven te tekenen. Gebruik voor beide datasets kolom `6-11yrs` als y-score en transformeer de kolom `y_true` in 1 (voor de waarden `6-11yrs` en een 0 voor de andere waarden).
    - Wat is de beste treshold-waarde voor elk van de ROC-curves?
    - Welk model heeft uw voorkeur?